In [3]:
import json
import os
from moviepy.editor import TextClip, concatenate_videoclips, CompositeVideoClip
from gtts import gTTS
from datetime import datetime
import subprocess

# Sample structured data
response = """
{
    "title": "Boat Chronicles: A Journey Through Maritime History",
    "segments": [
        {"start": 0, "duration": 6, "text": "From Rafts to Ocean Liners: Tracing the Evolution of Maritime Travel."},
        {"start": 7, "duration": 5, "text": "Explorers and Empires: Navigating the High Seas of Medieval Times."}
    ]
}
"""

data = json.loads(response)
title = data['title']
segments = data['segments']

# Create output directory
dir_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(dir_name, exist_ok=True)

def create_text_clip(segment):
    """Generate hacker-style green text clip."""
    return TextClip(segment['text'], font='Courier', fontsize=50, color='green', bg_color='black',
                    size=(1080, 1920)).set_duration(segment['duration']).set_start(segment['start'])

def generate_audio(segment, idx):
    """Generate audio using gTTS."""
    audio_path = os.path.join(dir_name, f"segment_{idx}.mp3")
    tts = gTTS(text=segment['text'], lang='en')
    tts.save(audio_path)
    return audio_path

# Generate text and audio clips
video_clips = [create_text_clip(seg) for seg in segments]
audio_paths = [generate_audio(seg, idx) for idx, seg in enumerate(segments)]

# Concatenate video clips
final_video_clip = concatenate_videoclips(video_clips, method="compose")
video_output_path = os.path.join(dir_name, "video_only.mp4")
final_video_clip.write_videofile(video_output_path, codec="libx264", fps=24)

# Combine audio and video using ffmpeg
final_audio = os.path.join(dir_name, "final_audio.mp3")
subprocess.run(['ffmpeg', '-y', '-i', 'concat:' + '|'.join(audio_paths), '-acodec', 'copy', final_audio])

final_video_path = os.path.join(dir_name, f"{title.replace(' ', '_')}_final.mp4")
subprocess.run([
    'ffmpeg', '-y', '-i', video_output_path, '-i', final_audio, '-c:v', 'copy', '-c:a', 'aac', '-strict', 'experimental',
    final_video_path
])

print(f"Final video created at {final_video_path}")


Moviepy - Building video 2024-05-04_01-41-53/video_only.mp4.
Moviepy - Writing video 2024-05-04_01-41-53/video_only.mp4



ffmpeg version 7.0 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 15.0.0 (clang-1500.3.9.4)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.0 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --e

Moviepy - Done !
Moviepy - video ready 2024-05-04_01-41-53/video_only.mp4
Final video created at 2024-05-04_01-41-53/Boat_Chronicles:_A_Journey_Through_Maritime_History_final.mp4


Output #0, mp4, to '2024-05-04_01-41-53/Boat_Chronicles:_A_Journey_Through_Maritime_History_final.mp4':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf61.1.100
  Stream #0:0(und): Video: h264 (High) (avc1 / 0x31637661), yuv420p(progressive), 1080x1920, q=2-31, 35 kb/s, 24 fps, 24 tbr, 12288 tbn (default)
      Metadata:
        handler_name    : VideoHandler
        vendor_id       : [0][0][0][0]
        encoder         : Lavc61.3.100 libx264
  Stream #0:1: Audio: aac (LC) (mp4a / 0x6134706D), 24000 Hz, mono, fltp, 69 kb/s
      Metadata:
        encoder         : Lavc61.3.100 aac
[out#0/mp4 @ 0x127e05680] video:47KiB audio:98KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 5.294793%
size=     153KiB time=00:00:10.91 bitrate= 114.9kbits/s speed= 142x    
[aac @ 0x127e0c0c0] Qavg: 3426.125


In [3]:
import json
import os
from vidgear.gears import WriteGear
import cv2
from gtts import gTTS
from datetime import datetime
import numpy as np  # Import NumPy here

# Sample structured data
response = """
{
    "title": "Boat Chronicles: A Journey Through Maritime History",
    "segments": [
        {"start": 0, "duration": 6, "text": "From Rafts to Ocean Liners: Tracing the Evolution of Maritime Travel."},
        {"start": 7, "duration": 5, "text": "Explorers and Empires: Navigating the High Seas of Medieval Times."}
    ]
}
"""

data = json.loads(response)
segments = data['segments']

# Create output directory
dir_name = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
os.makedirs(dir_name, exist_ok=True)

# Configuration for the output video
output_params = {"-vcodec": "libx264", "-crf": 0, "-preset": "fast"}
writer = WriteGear(output='output.mp4', compression_mode=True, logging=True, **output_params)

# Process each segment
for idx, segment in enumerate(segments):
    # Text-to-Speech for the segment
    tts = gTTS(text=segment['text'], lang='en')
    audio_path = os.path.join(dir_name, f"segment_{idx}.mp3")
    tts.save(audio_path)

    # Create a frame with text
    height, width = 1920, 1080
    blank_image = 255 * np.ones(shape=[height, width, 3], dtype=np.uint8)
    text = segment['text']
    font = cv2.FONT_HERSHEY_SIMPLEX
    cv2.putText(blank_image, text, (50, height // 2), font, 1, (0, 255, 0), 2)

    # Write frame to video with the duration of the segment
    for _ in range(int(segment['duration'] * 24)):  # Assuming 24 FPS
        writer.write(blank_image)

# Finalize video file
writer.close()

# Combine audio files into one and sync with video
final_audio_path = os.path.join(dir_name, "final_audio.mp3")
os.system(f"ffmpeg -y -i 'concat:{'|'.join([os.path.join(dir_name, f'segment_{i}.mp3') for i in range(len(segments))])}' -acodec copy {final_audio_path}")
os.system(f"ffmpeg -y -i output.mp4 -i {final_audio_path} -c:v copy -c:a aac -strict experimental final_output.mp4")

print("Final video with audio is ready.")


02:14:21 ::   WriteGear   ::  DEBUG   :: Output Parameters: `{'-vcodec': 'libx264', '-crf': 0, '-preset': 'fast'}`
02:14:21 ::   WriteGear   ::  DEBUG   :: Compression Mode is enabled therefore checking for valid FFmpeg executable.
02:14:21 ::   WriteGear   :: WARNING  :: Optional `-disable_ffmpeg_window` flag is only available on Windows OS with `logging=False`. Discarding!
02:14:21 ::    Helper     ::  DEBUG   :: Final FFmpeg Path: ffmpeg
02:14:22 ::    Helper     ::  DEBUG   :: FFmpeg validity Test Passed!
02:14:22 ::    Helper     ::  DEBUG   :: Found valid FFmpeg Version: `b'7.0'` installed on this system
02:14:22 ::   WriteGear   ::  DEBUG   :: Found valid FFmpeg executable: `ffmpeg`.
02:14:22 ::   WriteGear   ::  DEBUG   :: Compression Mode with FFmpeg backend is configured properly.
02:14:22 ::   WriteGear   ::  DEBUG   :: InputFrame => Height:1920 Width:1080 Channels:3 Datatype:uint8
02:14:22 ::   WriteGear   ::  DEBUG   :: Executing FFmpeg command: `ffmpeg -y -f rawvideo -vco

Final video with audio is ready.


[out#0/mp4 @ 0x1250063d0] video:51KiB audio:100KiB subtitle:0KiB other streams:0KiB global headers:0KiB muxing overhead: 3.839658%
size=     156KiB time=00:00:10.56 bitrate= 121.1kbits/s speed= 135x    
[aac @ 0x12500cdd0] Qavg: 4056.836
